In [1]:
!python --version

Python 3.11.13


In [2]:
!pip install python-dotenv evaluate rouge_score
!pip install -U datasets==2.17.0
# !pip install transformers==4.41.2 peft==0.11.1 accelerate==0.30.1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=13ec1ac89cc3517f118fb4c5e47b10bf46bc2719b02765d851862e39fae66e61
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are insta

In [3]:
!pip list

Package                               Version
------------------------------------- -------------------
absl-py                               1.4.0
accelerate                            1.9.0
aiofiles                              24.1.0
aiohappyeyeballs                      2.6.1
aiohttp                               3.12.14
aiosignal                             1.4.0
alabaster                             1.0.0
albucore                              0.0.24
albumentations                        2.0.8
ale-py                                0.11.2
altair                                5.5.0
annotated-types                       0.7.0
antlr4-python3-runtime                4.9.3
anyio                                 4.9.0
argon2-cffi                           25.1.0
argon2-cffi-bindings                  21.2.0
array_record                          0.7.2
arviz                                 0.22.0
astropy                               7.1.0
astropy-iers-data                     0.2025.7.21.0.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from datasets import load_dataset
from transformers import (
    GenerationConfig,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    TrainingArguments,
    Trainer,
)
from peft import get_peft_model, LoraConfig, TaskType, PeftModel, PeftConfig
import evaluate
import torch
import os
from dotenv import load_dotenv
from tqdm import tqdm
import json

In [6]:
os.listdir("/content/drive/MyDrive/python_projects/fine-tuning-LLMs")

['.env', 'outputs', 'logs', 'models']

In [7]:
env_path = "/content/drive/MyDrive/python_projects/fine-tuning-LLMs/.env"

# Load environment variables
load_dotenv(env_path)

True

In [8]:
model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
# Handle padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [10]:
ds = load_dataset("knkarthick/samsum")

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/download/streaming_download_manager.py:784: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


Generating validation split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/download/streaming_download_manager.py:784: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/download/streaming_download_manager.py:784: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


In [11]:
train_data, val_data, test_data = ds['train'], ds['validation'], ds['test']
ds = None

In [12]:
train_data.shape, val_data.shape, test_data.shape

((14732, 3), (818, 3), (819, 3))

In [13]:
train_data[0]

{'id': '13818513',
 'dialogue': "Amanda: I baked  cookies. Do you want some?\nJerry: Sure!\nAmanda: I'll bring you tomorrow :-)",
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}

# Full Fine Tuning

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


In [ ]:
print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [ ]:
train_small = train_data.shuffle(seed=42).select(range(10))
val_small = val_data.shuffle(seed=42).select(range(2))

In [ ]:
# def tokenize_with_prompt(example):
#     prompts = [
#         f"""
# summarize the below conversation

# {dialogue}

# summary:
# """ for dialogue in example["dialogue"]
#     ]

#     inputs = tokenizer(
#         prompts,
#         truncation=True,
#         padding="max_length",
#         max_length=512,
#         return_tensors="pt"  # you wanted this
#     )
#     targets = tokenizer(
#         example["summary"],
#         truncation=True,
#         padding="max_length",
#         max_length=128,
#         return_tensors="pt"
#     )

#     input_ids = inputs["input_ids"]
#     attention_mask = inputs["attention_mask"]
#     labels = targets["input_ids"]

#     # Mask padding tokens in labels with -100
#     labels[labels == tokenizer.pad_token_id] = -100

#     return {
#         "input_ids": input_ids,
#         "attention_mask": attention_mask,
#         "labels": labels
#     }

def tokenize_with_prompt(example):
    prompts = [
        f"""
summarize the below conversation

{dialogue}

summary:
""" for dialogue in example["dialogue"]
    ]

    # Tokenize inputs
    model_inputs = tokenizer(
        prompts,
        padding="max_length",
        truncation=True,
        max_length=512
    )

    # Tokenize labels
    labels = tokenizer(
        example["summary"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    labels_input_ids = []
    for label_seq in labels["input_ids"]:
        label_seq = [token_id if token_id != tokenizer.pad_token_id else -100 for token_id in label_seq]
        labels_input_ids.append(label_seq)

    # Return all necessary components
    example['input_ids'] = model_inputs['input_ids']
    example['attention_mask'] = model_inputs['attention_mask']
    example['labels'] = labels_input_ids

    return example



tokenized_train = train_data.map(tokenize_with_prompt, batched=True)
tokenized_val = val_data.map(tokenize_with_prompt, batched=True)

# tokenized_train = train_small.map(tokenize_with_prompt, batched=True)
# tokenized_val = val_small.map(tokenize_with_prompt, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
tokenized_train

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [ ]:
tokenized_train = tokenized_train.remove_columns(['id', 'dialogue', 'summary',])
tokenized_val = tokenized_val.remove_columns(['id', 'dialogue', 'summary',])

In [ ]:
tokenized_train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [ ]:
print(tokenized_train['input_ids'][0])

[21603, 8, 666, 3634, 21542, 10, 27, 13635, 5081, 5, 531, 25, 241, 128, 58, 16637, 10, 10625, 55, 21542, 10, 27, 31, 195, 830, 25, 5721, 3, 10, 18, 61, 9251, 10, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
print(tokenized_train['labels'][0])

[21542, 13635, 5081, 11, 56, 830, 16637, 128, 5721, 5, 1, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]


In [14]:
output_dir = "/content/drive/MyDrive/python_projects/fine-tuning-LLMs/models/flan-t5-full-samsum"
logging_dir = "/content/drive/MyDrive/python_projects/fine-tuning-LLMs/logs/flan-t5-full-samsum"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

In [ ]:
early_stop = EarlyStoppingCallback(
    early_stopping_patience=2,  # Stop if no improvement for 2 evals
    early_stopping_threshold=0.0001
)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    learning_rate=5e-4,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir=logging_dir,
    logging_strategy="epoch",
    # predict_with_generate=True,
    fp16=False, # False for GPU and True for CPU

    report_to="none",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)



In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True, return_tensors="pt")

In [ ]:
def get_latest_checkpoint(path):
    if not os.path.isdir(path):
        return None
    checkpoints = [os.path.join(path, d) for d in os.listdir(path) if d.startswith("checkpoint-")]
    return max(checkpoints, key=os.path.getmtime) if checkpoints else None

latest_checkpoint = get_latest_checkpoint(output_dir)
print(f'latest_checkpoint: {latest_checkpoint}')

latest_checkpoint: None


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    processing_class=tokenizer,
    data_collator=data_collator,
    callbacks=[early_stop]
)

In [ ]:
trainer.train(resume_from_checkpoint=latest_checkpoint)

Epoch,Training Loss,Validation Loss
1,1.693100,1.518342
2,1.298500,1.506663
3,1.009700,1.525893


Epoch,Training Loss,Validation Loss
1,1.693100,1.518342
2,1.298500,1.506663
3,1.009700,1.525893
4,0.777100,1.640740


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=14732, training_loss=1.1945963685291796, metrics={'train_runtime': 11184.8244, 'train_samples_per_second': 6.586, 'train_steps_per_second': 1.646, 'total_flos': 4.035138126505574e+16, 'train_loss': 1.1945963685291796, 'epoch': 4.0})

In [ ]:
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/python_projects/fine-tuning-LLMs/models/flan-t5-full-samsum/tokenizer_config.json',
 '/content/drive/MyDrive/python_projects/fine-tuning-LLMs/models/flan-t5-full-samsum/special_tokens_map.json',
 '/content/drive/MyDrive/python_projects/fine-tuning-LLMs/models/flan-t5-full-samsum/spiece.model',
 '/content/drive/MyDrive/python_projects/fine-tuning-LLMs/models/flan-t5-full-samsum/added_tokens.json',
 '/content/drive/MyDrive/python_projects/fine-tuning-LLMs/models/flan-t5-full-samsum/tokenizer.json')

In [16]:
tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(output_dir)

In [18]:
sample_indices = [10, 40, 765]
for i, sample_index in enumerate(sample_indices):
  dialogue = test_data[sample_index]['dialogue']
  summary = test_data[sample_index]['summary']
  prompt = f'''
summarize the below conversation

{dialogue}

summary:
  '''
  # Input constructed prompt instead of the dialogue.
  inputs = tokenizer(prompt, return_tensors='pt')
  output = tokenizer.decode(
      model.generate(
          input_ids = inputs["input_ids"],
          generation_config=GenerationConfig(max_new_tokens=50)
          # max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )
  print('#'*50)
  print('-'*50)
  print(f'Example {i+1}')
  print('-'*50)
  print(f'prompt:\n{prompt}')
  print('-'*50)
  print(f'summary:\n{summary}')
  print('-'*50)
  print(f'Full fine tuning model output:\n{output}')
  print('-'*50)
  print('#'*50)
  print()

##################################################
--------------------------------------------------
Example 1
--------------------------------------------------
prompt:

summarize the below conversation

Wanda: Let's make a party!
Gina: Why?
Wanda: beacuse. I want some fun!
Gina: ok, what do u need?
Wanda: 1st I need too make a list
Gina: noted and then?
Wanda: well, could u take yours father car and go do groceries with me?
Gina: don't know if he'll agree
Wanda: I know, but u can ask :)
Gina: I'll try but theres no promisess
Wanda: I know, u r the best!
Gina: When u wanna go
Wanda: Friday?
Gina: ok, I'll ask

summary:
  
--------------------------------------------------
summary:
Wanda wants to throw a party. She asks Gina to borrow her father's car and go do groceries together. They set the date for Friday. 
--------------------------------------------------
Full fine tuning model output:
Wanda and Gina are going to make a party on Friday.
------------------------------------------

In [19]:
with open('/content/drive/MyDrive/python_projects/fine-tuning-LLMs/outputs/test_data_json.json', 'r') as f:
  test_data_json = json.load(f)

In [20]:
test_data_json.keys()

dict_keys(['dialogue', 'summary', 'base_model'])

In [21]:
full_fine_tuned_model_outputs = []

for dialogue, summary in tqdm(zip(test_data_json['dialogue'], test_data_json['summary'])):
  prompt = f'''
summarize the below conversation

{dialogue}

summary:
  '''

  inputs = tokenizer(prompt, return_tensors='pt')
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )
  full_fine_tuned_model_outputs.append(output)

18it [01:03,  3.42s/it]Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors
819it [40:58,  3.00s/it]


In [22]:
test_data_json['full_fine_tuned_model'] = full_fine_tuned_model_outputs

test_data_json.keys()

dict_keys(['dialogue', 'summary', 'base_model', 'full_fine_tuned_model'])

In [23]:
with open('/content/drive/MyDrive/python_projects/fine-tuning-LLMs/outputs/test_data_json.json', 'w') as f:
  json.dump(test_data_json, f, indent=4)

In [24]:
with open('/content/drive/MyDrive/python_projects/fine-tuning-LLMs/outputs/test_data_json.json', 'r') as f:
  test_data_json = json.load(f)

In [25]:
# Load metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

In [26]:
rouge_result = rouge.compute(predictions=test_data_json['full_fine_tuned_model'], references=test_data_json['summary'])
bleu_result = bleu.compute(predictions=test_data_json['full_fine_tuned_model'], references=test_data_json['summary'])

In [27]:
rouge_result, bleu_result

({'rouge1': np.float64(0.4854452266868218),
  'rouge2': np.float64(0.24511834248744852),
  'rougeL': np.float64(0.405682805140837),
  'rougeLsum': np.float64(0.4052814356258132)},
 {'bleu': 0.17729537194613276,
  'precisions': [0.5165470251855457,
   0.24828734233945834,
   0.14554054054054055,
   0.08704670624418855],
  'brevity_penalty': 0.8830742200469339,
  'length_ratio': 0.8894059084514663,
  'translation_length': 16438,
  'reference_length': 18482})